In [1]:
training_data=[
    ['sunny','hot','high','weak','yes'],
    ['sunny','hot','high','strong','no'],
    ['overcast','hot','high','weak','maybe'],
    ['overcast','cool','normal','weak','no'],
    ['rain','mild','high','weak','maybe'],
    ['rain','cool','normal','weak','yes'],
    ['rain','cool','normal','strong','no'],
    ['overcast','cool','normal','weak','yes'],
    ['sunny','mild','high','weak','no'],
    ['sunny','hot','high','strong','yes'],
    ['sunny','cold','normal','weak','yes'],
    ['rain','mild','normal','strong','yes'],
    ['rain','cool','normal','strong','maybe'],
    ['rain','cool','normal','strong','yes'],
]

header=["outlook","temp","humidity","wind","play_tennis?"]

In [2]:
def class_counts(rows):
    count={}
    for row in rows:
        target=row[-1]
        if target not in count:
            count[target]=0
        count[target]+=1
    return count
class_counts(training_data)

{'maybe': 3, 'no': 4, 'yes': 7}

In [3]:
class Decision:
    

    def __init__(self, column, value):
        self.column = column
        self.value = value

    def match(self, example):
       
        val = example[self.column]
        return val == self.value

    def __repr__(self):
       
        condition = "=="
        return "Is %s %s %s?" % (
            header[self.column], condition, str(self.value))
Decision(0, "rain")

Is outlook == rain?

In [6]:
def partition(rows, question):
   
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows



In [8]:
true_rows, false_rows = partition(training_data, Decision(0,"sunny"))
true_rows

[['sunny', 'hot', 'high', 'weak', 'yes'],
 ['sunny', 'hot', 'high', 'strong', 'no'],
 ['sunny', 'mild', 'high', 'weak', 'no'],
 ['sunny', 'hot', 'high', 'strong', 'yes'],
 ['sunny', 'cold', 'normal', 'weak', 'yes']]

In [9]:
def gini(rows):
   
    counts = class_counts(rows)
    impurity = 1
    for tr in counts:
        prob_of_tr = counts[tr] / float(len(rows))
        impurity -= prob_of_tr**2
    return impurity

In [10]:
def info_gain(true, false, current_uncertainty):
   
    p = float(len(true)) / (len(true) + len(false))
    return current_uncertainty - p * gini(true) - (1 - p) * gini(false)

In [20]:
def find_best_split(rows):
  
    best_gain = 0  
    best_question = None  
    current_uncertainty = gini(rows)
    n_features = len(rows[0]) - 1 

    for col in range(n_features): 

        values = set([row[col] for row in rows])  

        for val in values: 

            question = Decision(col, val)

           
            true_rows, false_rows = partition(rows, question)

           
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue

          
            gain = info_gain(true_rows, false_rows, current_uncertainty)

           
            if gain >= best_gain:
                best_gain, best_question = gain, question

    return best_gain, best_question

In [21]:
class Leaf:
   
    def __init__(self, rows):
        self.predictions = class_counts(rows)

In [22]:
class Decision_Node:
   
    def __init__(self,question,true_branch,false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [23]:
def build_tree(rows):
   
    gain, question = find_best_split(rows)
    if gain == 0:
        return Leaf(rows)

    true_rows, false_rows = partition(rows, question)
    true_branch = build_tree(true_rows)
    false_branch = build_tree(false_rows)
    return Decision_Node(question, true_branch, false_branch)

In [24]:
def print_tree(node, spacing=""):
    if isinstance(node, Leaf):
        print (spacing + "Predict", node.predictions)
        return
    print (spacing + str(node.question))
    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")
    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")

In [26]:
my_tree = build_tree(training_data)
print_tree(my_tree)

Is outlook == sunny?
--> True:
  Is temp == mild?
  --> True:
    Predict {'no': 1}
  --> False:
    Is wind == weak?
    --> True:
      Predict {'yes': 2}
    --> False:
      Predict {'no': 1, 'yes': 1}
--> False:
  Is humidity == normal?
  --> True:
    Is temp == cool?
    --> True:
      Is wind == weak?
      --> True:
        Is outlook == overcast?
        --> True:
          Predict {'no': 1, 'yes': 1}
        --> False:
          Predict {'yes': 1}
      --> False:
        Predict {'no': 1, 'maybe': 1, 'yes': 1}
    --> False:
      Predict {'yes': 1}
  --> False:
    Predict {'maybe': 2}


In [27]:
def classify(row, node):
    if isinstance(node, Leaf):
        return node.predictions
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

In [28]:
 testing_data=[
    ['sunny','mild','normal','weak','no'],
    ['sunny','mild','normal','weak','maybe'],
    ['overcast','hot','high','high','maybe'],
     ['overcast','cool','normal','high','no'],
     ['rain','mild','high','strong','no'],
     ['rain','cool','normal','strong','no'],
     ['rain','cool','normal','weak','yes'],
]

In [29]:
def print_leaf(counts):
    total = sum(counts.values()) * 1.0
    probs = {}
    for tr in counts.keys():
        probs[tr] = str(int(counts[tr] / total * 100)) + "%"
    return probs

In [30]:
for row in testing_data:
    print ("Actual: %s. Predicted: %s" %
           (row[-1], print_leaf(classify(row, my_tree))))

Actual: no. Predicted: {'no': '100%'}
Actual: maybe. Predicted: {'no': '100%'}
Actual: maybe. Predicted: {'maybe': '100%'}
Actual: no. Predicted: {'no': '33%', 'maybe': '33%', 'yes': '33%'}
Actual: no. Predicted: {'maybe': '100%'}
Actual: no. Predicted: {'no': '33%', 'maybe': '33%', 'yes': '33%'}
Actual: yes. Predicted: {'yes': '100%'}
